# Welcome to the lab!

I'm writing this collection of Jupyter notebooks to be used as introductory materials for new undergrads. I will assume you know enough Python (should be familiar with object-oriented programming and whatever package manager / IDE you are using) as well as the first couple of chapters of *Molecular Mechanisms of Photosynthesis*, written by Blankenship. 

# Getting Started
## Biopython
The main library we will use extensively outside our own is the [Biopython library](https://biopython.org). This library comes with prepackaged tools for computational biology, especially for sequencing/alignment, population genetics, and structural biology. We focus primarily on the latter. To be brutally honest, the Biopython documentation sucks (what do you expect when scientists try to be software engineers?). Part of the motivation for this was to make this transition easier.
## PDB Files
To help in this regard, we will also use PDB ([Protein Data Bank](https://www.rcsb.org/#Category-learn)) files. These files contain atomic structures and are mainly used for biological molecules such as DNA/RNA and proteins. Specifically, atomic coordinates (i.e., the 3D arrangement of atoms involved) and bonding information are encoded. The database itself is also a worldwide collaboration effort. 
## Goal
**Our goal for the first tutorial is to learn how to grab data/information from PDB files using the Biopython library.**

# The Biopython PDB Module

## Reading the PDB File
In Biopython, you read a PDB file by creating a [`PDBParser` object](https://biopython.org/docs/1.75/api/Bio.PDB.PDBParser.html) that reads (hence, "parses") the file. Take note on the `PERMISSIVE` argument, which allows us to build the structure even if there might be errors in the file itself. We will also use the `get_structure` method to return a [`Structure` object](https://biopython.org/docs/1.75/api/Bio.PDB.Structure.html). We will be analyzing the LHCII Complex, which you should have already read about in Blankenship's book.

In [ ]:
from Bio.PDB.PDBParser import PDBParser

parser = PDBParser(PERMISSIVE=True)
structure = parser.get_structure("LHCII", "1rwt_trimer_opm.pdb")

## The `Structure` Model

Structures in Biopython follow a hierarchial model, the [SMCRA (Structure Model Chain Residue Atom) architecture](https://biopython.org/DIST/docs/tutorial/Tutorial.html#sec246): 
- A structure consists of models.
- A model consists of chains.
- A chain consists of residues.
- A residue consists of atoms.

This is representative of the way scientists think about protein structures. We can return models/chains/residues/atoms of a [`Structure` object](https://biopython.org/docs/1.75/api/Bio.PDB.Structure.html) by using the methods `get_models`, `get_chains`, `get_residues`, and `get_atoms` (as listed in the documentation). Using these respectively returns the [`Model`](https://biopython.org/docs/1.75/api/Bio.PDB.Model.html), [`Chain`](https://biopython.org/docs/1.75/api/Bio.PDB.Chain.html), [`Residue`](https://biopython.org/docs/1.75/api/Bio.PDB.Residue.html), and [`Atom`](https://biopython.org/docs/1.75/api/Bio.PDB.Model.html) objects as an iterable by Python generators. All of these also inherit the [`Entity`](https://biopython.org/docs/1.75/api/Bio.PDB.Entity.html) class. Read the documentation and familiarize yourself with the various things you can do with each of these objects. Specifically, you should pay extra attention to the [`Atom`](https://biopython.org/docs/1.75/api/Bio.PDB.Model.html) object and the following methods:

- `__sub__(self, other)`: subtract two `Atom` objects to return the distance between them.
- `get_coord`: Return the coordinates of an atom as a [NumPy `array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html).
- `get_name`: Return the name of an atom as a string. Note that in PDB files, an atom name is usually a 4-character label with spaces at the beginning and end. However, these spaces are often left out for simplicity. For instance, in a file, an amino acid C α atom might be labeled as ".CA." where the dots represent spaces. When creating an atom name (and its unique identifier or atom ID), we usually remove the spaces unless doing so would create a conflict.

## Homework Problem
Try a problem out to test your understanding. I will write *a* solution below, but please attempt it first.

The LHCII Complex contains a number of chlorophyll A and chlorophyll B molecules. Both of these kinds of molecules contain Mg atoms in the center of their porphyrin rings. Calculate the mean distance between the Mg atoms for every pair of chlorophylls in the structure. Use the fact that the atom name for magnesium is simply just `MG`. You may also use other libraries such as NumPy.


## Solution

The `for` loop for the distances relies on the summation: $$x_{\text{mean}} = \frac{1}{N} \sum^{N}_{i=1} \sum^{N}_{j > i} | x_{i} - x_{j} |. $$

In [3]:
from Bio.PDB.PDBParser import PDBParser
from numpy import mean 

parser = PDBParser(PERMISSIVE=True)
structure = parser.get_structure("LHCII", "1rwt_trimer_opm.pdb")

magnesium_atoms = []
for atom in structure.get_atoms():
    if atom.get_name() == "MG":
        magnesium_atoms.append(atom)

distances = []
for i in range(len(magnesium_atoms)):
    for j in range(i + 1, len(magnesium_atoms)):
        distance = magnesium_atoms[i] - magnesium_atoms[j]
        distances.append(distance)


mean_distance = mean(distances)
print(f"{mean_distance}")

35.429466247558594


# Challenge

As scientists, we are not freed from the burden of writing good code. Try seeing if you can optimize the solution above (or your solution) by paying note to time complexity of the double for loop. What time complexity is it in terms of big-O? What would be a better algorithm to use? 